<a href="https://colab.research.google.com/github/RogerHeederer/ML_practice_MyFirstML_B/blob/master/PretrainEmbedClassfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  %tensorflow_version 2.x
except Exception:
  pass

In [2]:
import os
import sys

import numpy as np
import pandas as pd
from numpy import array
from numpy import asarray
from numpy import zeros

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import models, regularizers, layers, optimizers, losses, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Flatten
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence

import tensorflow as tf
tf.random.set_seed(1)
np.random.seed(1)

In [3]:
paragraph_dict_list = [
         {'paragraph': 'dishplace is located in sunnyvale downtown there is parking around the area but it can be difficult to find during peak business hours my sisters and i came to this place for dinner on a weekday they were really busy so i highly recommended making reservations unless you have the patience to wait', 'category': 'food'},
         {'paragraph': 'service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations', 'category': 'food'},
         {'paragraph': 'portions are huge both french toast and their various omelettes are really good their french toast is probably 1.5x more than other brunch places great place to visit if you are hungry and dont want to wait 1 hour for a table', 'category': 'food'},
         {'paragraph': 'we started with apps going the chicken and waffle slides and chicken nachos the sliders were amazing and the nachos were good too maybe by themselves the nachos would have scored better but after those sliders they were up against some tough competition', 'category': 'food'},
         {'paragraph': 'the biscuits and gravy was too salty two people in my group had the gravy and all thought it was too salty my hubby ordered a side of double egg and it was served on two small plates who serves eggs to one person on separate plates we commented on that when it was delivered and even the server laughed and said she doesnt know why the kitchen does that presentation of food is important and they really missed on this one', 'category': 'food'},
         {'paragraph': 'the garlic fries were a great starter (and a happy hour special) the pancakes looked and tasted great and were a fairly generous portion', 'category': 'food'},
         {'paragraph': 'our meal was excellent i had the pasta ai formaggi which was so rich i didnt dare eat it all although i certainly wanted to excellent flavors with a great texture contrast between the soft pasta and the crisp bread crumbs too much sauce for me but a wonderful dish', 'category': 'food'},
         {'paragraph': 'what i enjoy most about palo alto is so many restaurants have dog-friendly seating outside i had bookmarked italico from when they first opened about a 1.5 years ago and was jonesing for some pasta so time to finally knock that bookmark off', 'category': 'food'},
         {'paragraph': 'the drinks came out fairly quickly a good two to three minutes after the orders were taken i expected my iced tea to taste a bit more sweet but this was straight up green tea with ice in it not to complain of course but i was pleasantly surprised', 'category': 'food'},
         {'paragraph': 'despite the not so good burger the service was so slow the restaurant wasnt even half full and they took very long from the moment we got seated to the time we left it was almost 2 hours we thought that it would be quick since we ordered as soon as we sat down my coworkers did seem to enjoy their beef burgers for those who eat beef however i will not be returning it is too expensive and extremely slow service', 'category': 'food'},
    
         {'paragraph': 'the four reigning major champions simona halep caroline wozniacki angelique kerber and defending us open champion sloane stephens could make a case for being the quartet most likely to succeed especially as all but stephens has also enjoyed the no1 ranking within the last 14 months as they prepare for their gruelling new york campaigns they currently hold the top four places in the ranks', 'category': 'sports'},
         {'paragraph': 'the briton was seeded nn7 here last year before a slump in form and confidence took her down to no46 after five first-round losses but there have been signs of a turnaround including a victory over a sub-par serena williams in san jose plus wins against jelena ostapenko and victoria azarenka in montreal. konta pulled out of new haven this week with illness but will hope for good things where she first scored wins in a major before her big breakthroughs to the semis in australia and wimbledon', 'category': 'sports'},
         {'paragraph': 'stephens surged her way back from injury in stunning style to win her first major here last year—and ranked just no83 she has since proved what a big time player she is winning the miami title via four fellow major champions then reaching the final at the french open back on north american hard courts she ran to the final in montreal only just edged out by halep she has also avoided many of the big names in her quarter—except for wild card azarenka as a possible in the third round', 'category': 'sports'},
         {'paragraph': 'when it came to england chances in the world cup it would be fair to say that most fans had never been more pessimistic than they were this year after enduring years of truly dismal performances at major tournaments – culminating in the 2014 event where they failed to win any of their three group games and finished in bottom spot those results led to the resignation of manager roy hodgson', 'category': 'sports'},
         {'paragraph': 'the team that eliminated russia – croatia – also improved enormously during the tournament before it began their odds were 33/1 but they played with real flair and star players like luka modric ivan rakitic and ivan perisic showed their quality on the world stage having displayed their potential by winning all three of their group stage games croatia went on to face difficult tests like the semi-final against england', 'category': 'sports'},
         {'paragraph': 'the perseyside outfit finished in fourth place in the premier league table and without a trophy last term after having reached the champions league final before losing to real madrid', 'category': 'sports'},
         {'paragraph': 'liverpool fc will return to premier league action on saturday lunchtime when they travel to leicester city in the top flight as they look to make it four wins in a row in the league', 'category': 'sports'},
         {'paragraph': 'alisson signed for liverpool fc from as roma this summer and the brazilian goalkeeper has helped the reds to keep three clean sheets in their first three premier league games', 'category': 'sports'},
         {'paragraph': 'but the rankings during that run-in to new york hid some very different undercurrents for murray had struggled with a hip injury since the clay swing and had not played a match since losing his quarter-final at wimbledon and he would pull out of the us open just two days before the tournament began—too late however to promote nederer to the no2 seeding', 'category': 'sports'},
         {'paragraph': 'then came the oh-so-familiar djokovic-nadal no-quarter-given battle for dominance in the third set there were exhilarating rallies with both chasing to the net both retrieving what looked like winning shots nadal more than once pulled off a reverse smash and had his chance to seal the tie-break but it was djokovic serving at 10-9 who dragged one decisive error from nadal for a two-sets lead', 'category': 'sports'}
]
df = pd.DataFrame(paragraph_dict_list)
df = df[['paragraph', 'category']]

In [4]:
df.head()

,paragraph,category
0,dishplace is located in sunnyvale downtown the...,food
1,service can be slower during busy hours but ou...,food
2,portions are huge both french toast and their ...,food
3,we started with apps going the chicken and waf...,food
4,the biscuits and gravy was too salty two peopl...,food


In [5]:
docs = df['paragraph'].tolist()
categories = df['category'].tolist()

In [12]:
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1
encoded_docs = t.texts_to_sequences(docs)

In [13]:
vocab_size

539

In [25]:
print(docs[1],'\n', encoded_docs[1])

service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations 
 [60, 91, 25, 181, 40, 93, 57, 9, 96, 182, 7, 183, 3, 184, 185, 43, 27, 186, 187]


In [32]:
def get_max_length(df):
    """
    데이터에서 가장 긴 문장의 단어 갯수를 리턴합니다.
    """
    max_length = 0
    for row in df['paragraph']:
        if len(row.split(" ")) > max_length:
            max_length = len(row.split(" "))
    return max_length

max_length = get_max_length(df)
print(max_length)

91


In [33]:
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [34]:
padded_docs[1] #max_length로 고정 벡터 박아두고 나머지는 padding 0 처리

array([ 60,  91,  25, 181,  40,  93,  57,   9,  96, 182,   7, 183,   3,
       184, 185,  43,  27, 186, 187,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
      dtype=int32)

In [35]:
def category_encode(category):
  if category == 'food':
    return [1,0]
  else:
    return [0,1]

labels = array([category_encode(category) for category in categories])

In [39]:
print(docs[1],'\n', encoded_docs[1], '\n', labels[1])

service can be slower during busy hours but our waiter was courteous and help gave some great entree recommendations 
 [60, 91, 25, 181, 40, 93, 57, 9, 96, 182, 7, 183, 3, 184, 185, 43, 27, 186, 187] 
 [1 0]


# Glove 워드 임베딩 다운로드

In [40]:
!wget -P . "http://nlp.stanford.edu/data/glove.6B.zip"

--2020-09-02 08:00:12--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-09-02 08:00:12--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-09-02 08:00:12--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘./glove.6B.zip’

glove

In [42]:
!unzip glove.6B.zip

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [100]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

# Glove 벡터로 실습 문장들의 단어를 인코딩합니다.
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [109]:
i, word, embeddings_index.get('lead'), embedding_vector, embedding_matrix[i]

(538,
 'lead',
 array([-0.25386  , -0.18414  ,  0.77174  , -0.40845  ,  0.54685  ,
        -0.11824  ,  0.3168   , -0.38241  , -0.77227  , -0.05886  ,
        -0.096802 ,  0.11445  , -0.53188  ,  0.28516  , -0.18447  ,
        -0.0059722, -0.10476  , -0.25732  , -0.45006  ,  0.089648 ,
         0.83928  , -0.43357  , -0.17928  ,  0.87369  ,  0.37506  ,
        -0.21013  , -0.21159  , -0.39015  ,  0.52559  , -0.53812  ,
        -0.71283  ,  0.48146  , -0.19663  ,  0.084202 , -0.19941  ,
        -0.71916  ,  0.23898  ,  0.47872  , -0.23237  ,  0.55953  ,
        -0.025114 , -0.35138  , -0.019317 , -0.18183  ,  0.57951  ,
        -0.1633   ,  0.017283 , -0.0097227, -0.65136  , -1.2412   ,
        -0.23302  , -0.3414   , -0.67617  ,  1.0098   ,  0.6952   ,
        -2.8957   , -0.53449  ,  0.10996  ,  0.86611  ,  0.67889  ,
        -0.81849  , -0.060675 ,  0.39015  ,  0.35857  ,  0.83608  ,
         0.20377  ,  0.39232  ,  0.34525  , -0.20694  , -0.11292  ,
        -0.29517  ,  0.48383  , -

In [101]:
# 모델 생성
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=max_length, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(2, activation='softmax'))
# 모델의 손실 함수 및 옵티마이저를 정합니다.
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# 모델 학습
model.fit(padded_docs, array(labels), epochs=20, verbose=1)

Epoch 1/20
1/1 [==============================] - 0s 2ms/step - loss: 0.6076 - accuracy: 0.7000
Epoch 2/20
1/1 [==============================] - 0s 4ms/step - loss: 0.3995 - accuracy: 0.8500
Epoch 3/20
1/1 [==============================] - 0s 1ms/step - loss: 0.1585 - accuracy: 1.0000
Epoch 4/20
1/1 [==============================] - 0s 2ms/step - loss: 0.0879 - accuracy: 1.0000
Epoch 5/20
1/1 [==============================] - 0s 1ms/step - loss: 0.0642 - accuracy: 1.0000
Epoch 6/20
1/1 [==============================] - 0s 3ms/step - loss: 0.0475 - accuracy: 1.0000
Epoch 7/20
1/1 [==============================] - 0s 2ms/step - loss: 0.0335 - accuracy: 1.0000
Epoch 8/20
1/1 [==============================] - 0s 2ms/step - loss: 0.0229 - accuracy: 1.0000
Epoch 9/20
1/1 [==============================] - 0s 2ms/step - loss: 0.0155 - accuracy: 1.0000
Epoch 10/20
1/1 [==============================] - 0s 2ms/step - loss: 0.0107 - accuracy: 1.0000
Epoch 11/20
1/1 [======================

In [98]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 91, 100)           53900     
_________________________________________________________________
flatten_1 (Flatten)          (None, 9100)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 18202     
Total params: 72,102
Trainable params: 18,202
Non-trainable params: 53,900
_________________________________________________________________


In [112]:
print(padded_docs[1], labels[1])

[ 60  91  25 181  40  93  57   9  96 182   7 183   3 184 185  43  27 186
 187   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0] [1 0]


In [123]:
model.predict(padded_docs[1:10])

array([[9.9445504e-01, 5.5449503e-03],
       [9.9898988e-01, 1.0101526e-03],
       [9.9975532e-01, 2.4465958e-04],
       [9.9999678e-01, 3.2330349e-06],
       [9.9431497e-01, 5.6849578e-03],
       [9.9994922e-01, 5.0735885e-05],
       [9.9947435e-01, 5.2564702e-04],
       [9.9988759e-01, 1.1242177e-04],
       [9.9999952e-01, 5.1466083e-07]], dtype=float32)

In [127]:
print(array(labels))

[[1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [1 0]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]
 [0 1]]
